## Use this notebook to create rivers_and_streams and trails rasters

Once this notebook has been run, return to the "Gathering Data and Creating Optimal Paths" notebook for final analysis.

In [2]:
# Add data from "Gathering Data and Creating Optimal Paths" notebook

%store -r east
%store -r west
%store -r north
%store -r south

print(east)
print(west)
print(north)
print(south)

%store -r folder

print(folder)

45.5
-109.56
46.4
-106.45
C:\Users\15612\Documents\GIS-5571\Final Project


In [ ]:
project = arcpy.mp.ArcGISProject("CURRENT")
m = project.listMaps("Map")[0]
spatial_ref = arcpy.SpatialReference(4326)
arcpy.env.extent = arcpy.Extent(east,north,west,south)
arcpy.env.cellSize = folder + '\Land Cover\land_cover.tif'

## Creation of rivers and streams raster dataset

In [ ]:
# Create flow accumulation raster dataset for building rivers and streams raster dataset

fill_raster = arcpy.sa.Fill('clipped_DEM')

flow_dir = arcpy.sa.FlowDirection(fill_raster)

flow_acc = arcpy.sa.FlowAccumulation(flow_dir)

Create the streams raster using the flow accumulation raster dataset

In [ ]:
# Make a streams raster using only flow accumulation values above 5000

stream = arcpy.sa.RasterCalculator(
    ['flow_acc'],
    ["x"],
    "x > 5000"
)

water_calc.save(\Water Calc Test\stream.tif')

m.addDataFromPath(r'C:\Users\15612\Documents\GIS-5571\Final Project\Water Calc Test\stream.tif')

In [3]:
# Reclassify all values of 0 to "NODATA"

streams_null = arcpy.sa.Reclassify(
    in_raster = 'streams.tif',
    reclass_field = 'Value',
    remap = arcpy.sa.RemapValue([[0,'NODATA']])
)

NameError: name 'arcpy' is not defined

In [ ]:
# Create a polyline from the 'streams_null' raster

streams = arcpy.conversion.RasterToPolyline(
    in_raster = 'streams_null',
    out_polyline_features = 'streams'
)

In [ ]:
# Create a buffer around the polyline of 7.5 meters for the streams dataset

streams_buf = arcpy.analysis.Buffer(
    in_features = 'streams',
    out_feature_class = 'streams_buf',
    buffer_distance_or_field = '7.5 Meters'
)

In [ ]:
# Create a raster from the buffer feature

arcpy.conversion.PolygonToRaster(
    in_features = 'streams_buf',
    value_field = 'grid_code',
    out_rasterdataset = 'streams_wide'
)

In [ ]:
# Reclassify the 'streams_wide' raster to change "NODATA" values to 1 and values of 1 to 10

streams_wide_reclass = arcpy.sa.Reclassify(
    in_raster = 'streams_wide',
    reclass_field = 'Value',
    remap = arcpy.sa.RemapValue([['NODATA',1],[1,10]])
)

Create the rivers raster dataset using the flow accumulation raster dataset

In [ ]:
# Make a rivers raster utilizing only flow accumulation values above 500000

river = arcpy.sa.RasterCalculator(
    ['flow_acc'],
    ["x"],
    "x > 500000"
)

river.save(r'C:\Users\15612\Documents\GIS-5571\Final Project\Water Calc Test\river.tif')

m.addDataFromPath(r'C:\Users\15612\Documents\GIS-5571\Final Project\Water Calc Test\river.tif')

In [ ]:
# Reclassify all values of 0 to "NODATA"

river_null = arcpy.sa.Reclassify(
    in_raster = river,
    reclass_field = 'Value',
    remap = arcpy.sa.RemapValue([[0,'NODATA']])
)

In [ ]:
# Create a polyline feature from the 'river_null' raster dataset

rivers = arcpy.conversion.RasterToPolyline(
    in_raster = 'river_null.tif',
    out_polyline_features = 'rivers'
)

In [ ]:
# Create a buffer around the polyline feature class of 15 meters

rivers_buf = arcpy.analysis.Buffer(
    in_features = 'rivers',
    out_feature_class = 'rivers_buf',
    buffer_distance_or_field = '15 Meters'
)

In [ ]:
# Create a raster from the 15-meter buffer feature class

arcpy.conversion.PolygonToRaster(
    in_features = 'rivers_buf',
    value_field = 'grid_code',
    out_rasterdataset = 'rivers_wide'
)

In [ ]:
# Reclassify the 'rivers_wide' raster dataset to change "NODATA" values to 1 and values of 1 to 50

rivers_final = arcpy.sa.Reclassify(
    in_raster = 'rivers_wide',
    reclass_field = 'Value',
    remap = arcpy.sa.RemapValue([['NODATA',1],[1,50]])
)

Combine the rivers and streams raster into one

In [ ]:
# Combine the streams and rivers raster datasets together to form the final rivers_and_streams dataset

rivers_and_streams = arcpy.sa.Con(
    in_conditional_raster = 'rivers_final',
    in_true_raster_or_constant = 'streams_wide_reclass',
    in_false_raster_or_constant = 'rivers_final',
    where_clause = '"Value" = 1'
)

Create the trails raster dataset

In [ ]:
# Convert polyline trail data to a raster dataset

trails = arcpy.conversion.PolylineToRaster(
    in_features = 'vec_trail_clip',
    value_field = 'OBJECTID',
    out_rasterdataset = 'trails'
)

In [ ]:
# Iterate through each value in the created 'trails' raster to find the number of trails to reclassify

num = 0
with arcpy.da.SearchCursor('trails','Value') as cursor:
    for row in cursor:
        num += 1
        
#print(num)

In [ ]:
# Reclassify all values not equal to "NODATA" to 1 and all other values equal to 1

trails_reclass = arcpy.sa.Reclassify(
    in_raster = 'trails',
    reclass_field = 'Value',
    remap = arcpy.sa.RemapRange([[1,num,1],['NODATA',0]])
)